# Rolling dice markov chain example
$win = 1/6$

$roll\ again = 1/3$

$pass = 1/2$

Each row coordinates with the beginning position
Each colomn coordinates with the end position
$$ P =
\begin{bmatrix}
p_{11} & p_{12} & p_{11'} & p_{12'}\\
p_{21} & p_{22} & p_{21'} & p_{22'}\\
p_{1'1} & p_{1'2} & p_{1'1'} & p_{1'2'}\\
p_{2'1} & p_{2'2} & p_{2'1'} & p_{2'2'}
\end{bmatrix}
=
\begin{bmatrix}
roll\ again & pass & win & 0\\
pass & roll\ again & 0 & win\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{bmatrix}
$$

$p_{11'}(n)$ is the probability that player 1 will win given n rolls.

![](https://cdn.mathpix.com/snip/images/hICl4OF_2WACgnl0M5gvYMc5lDP5jor7rNQTlSC7M54.original.fullsize.png)


# Project Start